# Temporal Patterns Analysis with Neural Heatmap API

This notebook demonstrates time series analysis and temporal pattern detection.

## What You'll Learn
- Retrieve temporal pattern data
- Visualize time series of neural activity
- Detect periodic patterns and trends
- Analyze state changes in neural behavior

In [ ]:
# Import required libraries
import asyncio
from datetime import datetime
from pathlib import Path

from neural_heatmap import NeuralHeatmapClient, connect, TemporalPattern
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("Libraries loaded successfully!")

## 1. Connect to Server

In [ ]:
async def connect_to_server():
    client = await connect("http://localhost:8080")
    if client.connected:
        print("✅ Connected to Neural Heatmap server")
    return client

client = await connect_to_server()

## 2. Get Temporal Patterns

Retrieve all detected temporal patterns.

In [ ]:
async def get_all_temporal_patterns():
    # Get all temporal patterns
    patterns = await client.get_temporal_patterns()
    
    print(f"Found {len(patterns)} temporal patterns")
    
    # Group by pattern type
    pattern_types = {}
    for pattern in patterns:
        if pattern.pattern_type not in pattern_types:
            pattern_types[pattern.pattern_type] = []
        pattern_types[pattern.pattern_type].append(pattern)
    
    print("\nPattern Types:")
    for ptype, plist in pattern_types.items():
        print(f"  {ptype}: {len(plist)} patterns")
    
    return patterns, pattern_types

temporal_patterns, pattern_types = await get_all_temporal_patterns()

## 3. Analyze Periodic Patterns

Examine periodic patterns in neural activity.

In [ ]:
def analyze_periodic_patterns(patterns):
    """Analyze and display periodic patterns"""
    periodic = [p for p in patterns if p.pattern_type == 'periodic']
    
    if not periodic:
        print("No periodic patterns found")
        return
    
    print(f"\n{'Frequency (Hz)':<15} {'Amplitude':<12} {'Phase':<10} {'Confidence':<12}")
    print("-" * 60)
    
    for p in sorted(periodic, key=lambda x: x.frequency or 0):
        freq = f"{p.frequency:.3f}" if p.frequency else "N/A"
        amp = f"{p.amplitude:.3f}" if p.amplitude else "N/A"
        phase = f"{p.phase:.2f}" if p.phase else "N/A"
        conf = f"{p.confidence:.2%}" if p.confidence else "N/A"
        print(f"{freq:<15} {amp:<12} {phase:<10} {conf:<12}")
    
    return periodic

periodic_patterns = analyze_periodic_patterns(temporal_patterns)

## 4. Visualize Frequency Spectrum

Create a frequency spectrum of periodic patterns.

In [ ]:
def plot_frequency_spectrum(patterns):
    """Plot frequency spectrum of periodic patterns"""
    periodic = [p for p in patterns if p.pattern_type == 'periodic' and p.frequency]
    
    if not periodic:
        print("No periodic patterns with frequency data")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Frequency vs Amplitude scatter
    frequencies = [p.frequency for p in periodic]
    amplitudes = [p.amplitude for p in periodic if p.amplitude]
    confidences = [p.confidence for p in periodic]
    
    scatter = ax1.scatter(frequencies, amplitudes, c=confidences, 
                          cmap='viridis', s=100, alpha=0.7, edgecolors='black')
    ax1.set_xlabel('Frequency (Hz)', fontsize=12)
    ax1.set_ylabel('Amplitude', fontsize=12)
    ax1.set_title('Frequency vs Amplitude', fontsize=14)
    ax1.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax1, label='Confidence')
    
    # Frequency histogram
    ax2.hist(frequencies, bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    ax2.set_xlabel('Frequency (Hz)', fontsize=12)
    ax2.set_ylabel('Count', fontsize=12)
    ax2.set_title('Frequency Distribution', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_frequency_spectrum(temporal_patterns)

## 5. Analyze Trends

Examine trending patterns in neural activity.

In [ ]:
def analyze_trend_patterns(patterns):
    """Analyze and display trend patterns"""
    trends = [p for p in patterns if p.pattern_type == 'trend']
    
    if not trends:
        print("No trend patterns found")
        return
    
    print(f"\nFound {len(trends)} trend patterns")
    
    for i, trend in enumerate(trends[:10]):
        print(f"\nTrend {i+1}:")
        print(f"  Confidence: {trend.confidence:.2%}")
        if trend.start_time and trend.end_time:
            duration = trend.end_time - trend.start_time
            print(f"  Duration: {duration/1000:.2f} seconds")
        if trend.metadata:
            print(f"  Direction: {trend.metadata.get('direction', 'unknown')}")
    
    return trends

trend_patterns = analyze_trend_patterns(temporal_patterns)

## 6. Analyze State Changes

Examine state changes in neural behavior.

In [ ]:
def analyze_state_changes(patterns):
    """Analyze and display state change patterns"""
    state_changes = [p for p in patterns if p.pattern_type == 'state_change']
    
    if not state_changes:
        print("No state change patterns found")
        return
    
    print(f"\nFound {len(state_changes)} state changes")
    
    # Group by state type
    state_types = {}
    for sc in state_changes:
        state_type = sc.metadata.get('from_state', 'unknown')
        if state_type not in state_types:
            state_types[state_type] = []
        state_types[state_type].append(sc)
    
    print("\nState Transitions:")
    for state_type, changes in state_types.items():
        print(f"  {state_type}: {len(changes)} transitions")
    
    return state_changes

state_changes = analyze_state_changes(temporal_patterns)

## 7. Get Temporal Statistics

Retrieve comprehensive temporal statistics.

In [ ]:
async def get_temporal_stats():
    # Get temporal statistics
    stats = await client.get_temporal_statistics()
    
    print("Temporal Statistics:")
    print(f"  Total events: {stats.get('total_events', 'N/A')}")
    print(f"  Time span: {stats.get('time_span_ms', 'N/A')} ms")
    print(f"  Average event rate: {stats.get('avg_event_rate', 'N/A')} events/sec")
    print(f"  Patterns detected: {stats.get('patterns_detected', 'N/A')}")
    print(f"  State changes: {stats.get('state_changes', 'N/A')}")
    
    return stats

temporal_stats = await get_temporal_stats()

## 8. Visualize Temporal Timeline

Create a timeline visualization of all patterns.

In [ ]:
def plot_temporal_timeline(patterns, stats):
    """Create timeline visualization of temporal patterns"""
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Group patterns by type
    type_colors = {
        'periodic': '#1f77b4',
        'trend': '#ff7f0e',
        'state_change': '#2ca02c'
    }
    
    y_offset = 0
    legend_handles = []
    
    for ptype, color in type_colors.items():
        type_patterns = [p for p in patterns if p.pattern_type == ptype]
        
        if type_patterns:
            # Plot pattern occurrences
            for pattern in type_patterns:
                if pattern.start_time:
                    start = pattern.start_time / 1000  # Convert to seconds
                    duration = ((pattern.end_time or pattern.start_time) - pattern.start_time) / 1000
                    
                    ax.barh(y_offset, duration, left=start, height=0.5, 
                           color=color, alpha=0.7, edgecolor='black')
            
            # Add to legend
            from matplotlib.patches import Patch
            legend_handles.append(Patch(color=color, label=ptype.replace('_', ' ').title()))
            
            y_offset += 1
    
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_yticks(range(len(type_colors)))
    ax.set_yticklabels([t.replace('_', ' ').title() for t in type_colors.keys()])
    ax.set_title('Temporal Pattern Timeline', fontsize=14)
    ax.legend(handles=legend_handles, loc='upper right')
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

plot_temporal_timeline(temporal_patterns, temporal_stats)

## 9. Time Series Decomposition

Decompose temporal patterns into trend, seasonal, and residual components.

In [ ]:
try:
    from statsmodels.tsa.seasonal import seasonal_decompose
    
    def decompose_temporal_data(patterns, period=10):
        """Decompose temporal data into trend, seasonal, and residual"""
        # Create synthetic time series from patterns
        # (In real usage, you'd get actual time series data)
        periodic = [p for p in patterns if p.pattern_type == 'periodic']
        
        if not periodic:
            print("No periodic patterns for decomposition")
            return
        
        # Create synthetic time series
        t = np.linspace(0, 100, 1000)
        signal = np.zeros_like(t)
        
        for p in periodic[:5]:  # Use first 5 patterns
            if p.frequency and p.amplitude:
                phase = p.phase or 0
                signal += p.amplitude * np.sin(2 * np.pi * p.frequency * t + phase)
        
        # Add some noise
        signal += np.random.normal(0, 0.1, len(t))
        
        # Create time series
        ts = pd.Series(signal, index=pd.date_range('2024-01-01', periods=len(t), freq='S'))
        
        # Perform decomposition
        decomposition = seasonal_decompose(ts, model='additive', period=period)
        
        # Plot decomposition
        fig, axes = plt.subplots(4, 1, figsize=(14, 10))
        
        decomposition.observed.plot(ax=axes[0], title='Observed')
        decomposition.trend.plot(ax=axes[1], title='Trend')
        decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
        decomposition.resid.plot(ax=axes[3], title='Residual')
        
        plt.tight_layout()
        plt.show()
        
        return decomposition
    
    decomposition = decompose_temporal_data(temporal_patterns)
    
except ImportError:
    print("statsmodels not installed. Install with: pip install statsmodels")

## 10. Export Temporal Data

In [ ]:
async def export_temporal_data():
    """Export temporal pattern data"""
    export_dir = Path("./temporal_analysis")
    export_dir.mkdir(exist_ok=True)
    
    # Export patterns
    patterns_data = [
        {
            "type": p.pattern_type,
            "frequency": p.frequency,
            "amplitude": p.amplitude,
            "phase": p.phase,
            "confidence": p.confidence,
            "start_time": p.start_time,
            "end_time": p.end_time,
            "metadata": p.metadata
        }
        for p in temporal_patterns
    ]
    
    import json
    with open(export_dir / "temporal_patterns.json", "w") as f:
        json.dump(patterns_data, f, indent=2)
    print("✅ Exported temporal_patterns.json")
    
    # Export statistics
    with open(export_dir / "temporal_stats.json", "w") as f:
        json.dump(temporal_stats, f, indent=2)
    print("✅ Exported temporal_stats.json")

await export_temporal_data()

## 11. Cleanup

In [ ]:
async def cleanup():
    await client.disconnect()
    print("✅ Disconnected from server")

await cleanup()

## Summary

In this notebook, you learned:
- How to retrieve temporal pattern data
- Analyzing periodic patterns and frequency spectra
- Examining trends and state changes
- Visualizing temporal timelines
- Time series decomposition techniques

## Next Steps
- `04_anomaly_detection.ipynb` - Detect and analyze anomalies
- `05_custom_workflows.ipynb` - Build custom analysis workflows